## Deliverable 3. Create a Travel Itinerary Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [12]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Vaini,TO,87.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
1,5,Hithadhoo,MV,83.12,overcast clouds,-0.6000,73.0833,Scoop Guest House
2,6,Cayenne,GF,81.09,few clouds,4.9333,-52.3333,Hôtel Le Dronmi
3,11,Puerto Ayora,EC,77.81,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
4,14,Dingle,PH,79.00,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [13]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [14]:

# 4a. Add a marker layer for each city to the map.

max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 4b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start_df= vacation_df.loc[(vacation_df["City"] == "Saint George")]
vacation_end_df= vacation_df.loc[(vacation_df["City"] == "Saint George")]
vacation_stop1_df = vacation_df.loc[(vacation_df["City"] == "Lompoc")]
vacation_stop2_df = vacation_df.loc[(vacation_df["City"] == "San Jose")]
vacation_stop3_df= vacation_df.loc[(vacation_df["City"] == "Laguna")]

In [16]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start_df["Lat"].to_numpy()[0], vacation_start_df["Lng"].to_numpy()[0]
end = vacation_end_df["Lat"].to_numpy()[0], vacation_end_df["Lng"].to_numpy()[0]
stop1 = vacation_stop1_df["Lat"].to_numpy()[0], vacation_stop1_df["Lng"].to_numpy()[0]
stop2 = vacation_stop2_df["Lat"].to_numpy()[0], vacation_stop2_df["Lng"].to_numpy()[0]
stop3 = vacation_stop3_df["Lat"].to_numpy()[0], vacation_stop2_df["Lng"].to_numpy()[0]
print(f'Start:{start},End:{end}')
print(f'Stop 1: {stop1}, Stop 2: {stop2},Stop 3:{stop3}')


Start:(37.1041, -113.5841),End:(37.1041, -113.5841)
Stop 1: (34.6391, -120.4579), Stop 2: (37.3394, -121.895),Stop 3:(38.421, -121.895)


In [17]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary  = gmaps.directions_layer(
        start, end, waypoints=[stop1,stop2,stop3],
        travel_mode='DRIVING')
fig.add_layer(city_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [18]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start_df, vacation_stop1_df, vacation_stop2_df, vacation_stop3_df, vacation_end_df],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,63,Saint George,US,85.24,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
1,96,Lompoc,US,87.78,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
2,676,San Jose,US,85.28,broken clouds,37.3394,-121.8950,The Westin San Jose
3,443,Laguna,US,89.89,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
4,63,Saint George,US,85.24,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn


In [19]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,37.1041,-113.5841
1,34.6391,-120.4579
2,37.3394,-121.8950
3,38.4210,-121.4238
4,37.1041,-113.5841


In [20]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(-15.0, -50.0), zoom_level=3.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))